# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst. They will need to use statistical modeling and machine learning to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: 
{
  "coworker": "Data Analyst",
  "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
  "context": "The goal is to identify trends and predict market movements for AAPL. Make sure to provide insights and alerts about significant market opportunities or threats."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL and use statistical modeling a

Final Answer: 
Based on the real-time market data for AAPL obtained from Yahoo Finance, the stock price for Apple Inc. (AAPL) is currently at $224.26 with a 0.23% increase. The stock has shown a 5.20% increase in the last 5 days, a 30.96% increase in the last year, and a remarkable 264.53% increase in the last 5 years. The market cap stands at 3.361T with a forward dividend yield of 0.45%. The company's earnings date is scheduled between April 30, 2025, and May 5, 2025. Additionally, Apple Inc. offers a range of products and services in the consumer electronics sector, including smartphones, personal computers, tablets, wearables, and accessories. The company also provides various subscription-based services such as Apple Arcade, Apple Fitness+, Apple Music, Apple News+, and Apple TV+. 

Overall, based on the current market data and historical performance, Apple Inc. (AAPL) appears to be a strong contender for investment with its consistent growth and diverse product offerings in the t

Final Answer: Apple Inc. (AAPL) is currently trading at $224.26, with a percentage increase of 0.23%. The market cap is at $3.361T, with a forward dividend yield of 0.45%. The earnings date is set between April 30, 2025, and May 5, 2025. Apple Inc. is a technology company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. They offer various subscription-based services like Apple Arcade, Apple Music, and Apple TV+. The stock has seen a 1-year return of 30.96% and a 5-year return of 264.53%.

> Finished chain.
 

Apple Inc. (AAPL) is currently trading at $224.26, with a percentage increase of 0.23%. The market cap is at $3.361T, with a forward dividend yield of 0.45%. The earnings date is set between April 30, 2025, and May 5, 2025. Apple Inc. is a technology company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. They offer various subscription-bas

Thought: I now have gathered information on AAPL's stock performance, market outlook, the user's risk tolerance, and preference for Day Trading. 

Final Answer: 
Based on the information gathered, AAPL's stock performance shows a positive trend with analysts forecasting price targets ranging from $188 to $325. The stock has shown an increase in price and is currently trading around $223.75. The user's risk tolerance and preference for Day Trading should be aligned with their individual preferences, understanding of the market, and willingness to tolerate potential losses. It is essential for the user to explore different day trading strategies, signals, and options trading to maximize success while considering concentration risk and risk tolerance. It is recommended to use a broker for purchasing AAPL stocks and to explore various trading strategies to capitalize on market opportunities.

> Finished chain.
 

Based on the information gathered, AAPL's stock performance shows a positive 

I need to gather detailed risk analysis and mitigation strategies for the proposed trading strategies for AAPL. I should use the tools available to delegate tasks and ask questions to ensure a comprehensive report.

Action: Delegate work to co-worker
Action Input: {"coworker": "Risk Advisor", "task": "Conduct detailed risk analysis and suggest mitigation strategies for the proposed trading strategies for AAPL", "context": "The potential trading strategies for AAPL are Day Trading with Options, Technical Analysis-Based Trading, Sector Rotation Strategy, and Earnings-Based Trading. These strategies align with the user's risk tolerance and preference for Day Trading."}

> Entering new CrewAgentExecutor chain...
I need to gather information on the potential risks associated with the proposed trading strategies for AAPL and suggest appropriate mitigation strategies based on my analysis.

Action: Read website content
Action Input: {"website_url": "https://www.investopedia.com/trading/day-tra

Final Answer: I apologize for the inconvenience. Unfortunately, I was unable to access the detailed risk analysis and mitigation strategies for the proposed trading strategies for AAPL directly from the Risk Advisor. However, I will conduct further research and analysis on each of the trading strategies - Day Trading with Options, Technical Analysis-Based Trading, Sector Rotation Strategy, and Earnings-Based Trading - to provide a comprehensive risk analysis report with detailed potential risks and mitigation recommendations. Thank you for your understanding.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: I apologize for the inconvenience. Unfortunately, I was unable to access the detailed risk analysis and mitigation strategies for the proposed trading strategies for AAPL directly from the Risk Advisor. However, I will conduct further research and analysis on each of the trading strategies - Day Trading with Options, Technical Analysis-Based Trading, Sector Rotation Strategy

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

I apologize for the inconvenience. Unfortunately, I was unable to access the detailed risk analysis and mitigation strategies for the proposed trading strategies for AAPL directly from the Risk Advisor. However, I will conduct further research and analysis on each of the trading strategies - Day Trading with Options, Technical Analysis-Based Trading, Sector Rotation Strategy, and Earnings-Based Trading - to provide a comprehensive risk analysis report with detailed potential risks and mitigation recommendations. Thank you for your understanding.